<a href="https://colab.research.google.com/github/buriro-ezekia/Data-Analyst-Volunteer-Tasks/blob/main/Data_Cleaning_and_Preprocessing_Sleep_Habits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
# Import Necessary Libraries
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Load dataset
# The error is likely due to the encoding of the CSV file.
# We'll try to read it using 'latin-1' encoding.
# If this doesn't work, you may need to experiment with other encodings like 'cp1252', 'utf-16', etc.
df = pd.read_csv('drive/MyDrive/Data Analysis with Python/Apple Search Ads Campaigns.csv', encoding='latin-1')

print(df.head(10))

    Campaign ID                                  Campaign Name Start Date  \
0  4.703112e+08                         0.54 - May 2022 - Arab  31-Aug-20   
1  4.816894e+08                   0.54 - Tier 1 & 2 - May 2022  04-Oct-20   
2  4.699278e+08                       0.54 - Tier 1 - May 2022  29-Aug-20   
3  4.742696e+08                             1.00 - My Keywords  14-Sep-20   
4  4.840063e+08  1.01 - US/OZ - May 2022 - Performing Keywords  12-Oct-20   
5  4.870115e+08                          1.50 - May 2022 - USA  18-Oct-20   
6  5.798608e+08          1.50 - USA - May 2022 - Apple Suggest  03-Jul-21   
7  4.814068e+08                            Japan Oct 2020 0.19  04-Oct-20   
8  1.454603e+09                  Sleep - Exploration Sept 2023  16-Sep-23   
9  1.153490e+09                   Sleep Habits - Product pages  28-Sep-22   

   Status                    App Name                  Ad Placement  \
0  PAUSED  Sleep Habits: Sleep Better      App Store Search Results   
1  PAUSED 

In [49]:
# View Column names
print(df.columns)

Index(['Campaign ID', 'Campaign Name', 'Start Date', 'Status', 'App Name',
       'Ad Placement', 'Lifetime Budget', 'Daily Budget', 'Spend',
       'Average cost per tap (CPT)', 'Average cost per acquisition (CPA)',
       'Average Cost per Thousand-Impressions (CPM)', 'Impressions', 'Taps',
       'Installs', 'The tap-through rate (TTR)', 'Conversion Rate (CR)',
       'LAT On Installs', 'LAT Off Installs', 'New Downloads', 'Redownloads'],
      dtype='object')


In [50]:
# View information in the dataset
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 21 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Campaign ID                                  13 non-null     float64
 1   Campaign Name                                13 non-null     object 
 2   Start Date                                   13 non-null     object 
 3   Status                                       13 non-null     object 
 4   App Name                                     13 non-null     object 
 5   Ad Placement                                 13 non-null     object 
 6   Lifetime Budget                              10 non-null     float64
 7   Daily Budget                                 13 non-null     float64
 8   Spend                                        14 non-null     float64
 9   Average cost per tap (CPT)                   14 non-null     float64
 10  Aver

In [51]:
# Check for missing values
print(df.isnull().sum())

Campaign ID                                    1
Campaign Name                                  1
Start Date                                     1
Status                                         1
App Name                                       1
Ad Placement                                   1
Lifetime Budget                                4
Daily Budget                                   1
Spend                                          0
Average cost per tap (CPT)                     0
Average cost per acquisition (CPA)             0
Average Cost per Thousand-Impressions (CPM)    0
Impressions                                    0
Taps                                           0
Installs                                       0
The tap-through rate (TTR)                     0
Conversion Rate (CR)                           0
LAT On Installs                                0
LAT Off Installs                               0
New Downloads                                  0
Redownloads         

In [52]:
# Check for missing values in specific columns
critical_columns = ['Campaign ID', 'Campaign Name', 'Start Date', 'Status', 'App Name', 'Ad Placement', 'Lifetime Budget', 'Daily Budget']
missing_values_critical = df[critical_columns].isnull().sum()

print("Missing Values in Critical Columns:\n", missing_values_critical)

# Handle missing values (example: imputation with median for numerical columns)
for col in critical_columns:
    if pd.api.types.is_numeric_dtype(df[col]):
        df[col].fillna(df[col].median(), inplace=True)  # Impute with the median
    # Improved handling for other data types:
    elif pd.api.types.is_string_dtype(df[col]):
        # Use the most frequent value (mode) to fill missing values in string columns
        df[col].fillna(df[col].mode()[0], inplace=True)
    # Handle datetime objects
    elif pd.api.types.is_datetime64_any_dtype(df[col]):
        df[col].fillna(method='ffill', inplace=True)  # Forward fill for datetime columns
    else:
        # For other data types (if any), use forward fill
        df[col].fillna(method='ffill', inplace=True)


#Re-check for missing values after imputation
missing_values_after_imputation = df[critical_columns].isnull().sum()
print("\nMissing Values After Imputation:\n", missing_values_after_imputation)


Missing Values in Critical Columns:
 Campaign ID        1
Campaign Name      1
Start Date         1
Status             1
App Name           1
Ad Placement       1
Lifetime Budget    4
Daily Budget       1
dtype: int64

Missing Values After Imputation:
 Campaign ID        0
Campaign Name      0
Start Date         0
Status             0
App Name           0
Ad Placement       0
Lifetime Budget    0
Daily Budget       0
dtype: int64


In [53]:
# # Check for missing values after imputation now

# print(df.isnull().sum())

In [54]:
# Data Type Validation
# Ensure that the Start Date column is in a proper date format.
try:
  df['Start Date'] = pd.to_datetime(df['Start Date'], errors='raise')
  print("Start Date column successfully converted to datetime.")
except ValueError as e:
  print(f"Error converting 'Start Date' to datetime: {e}")
  # Handle the error appropriately, e.g., investigate the problematic rows,
  # try different date formats, or drop the rows if they're insignificant.
  # Example:
  # problematic_rows = df[pd.to_datetime(df['Start Date'], errors='coerce').isnull()]
  # print("Problematic rows:\n", problematic_rows)
  # df = df.dropna(subset=['Start Date'])


# Numeric Columns: Verify that columns such as Lifetime Budget, Daily Budget, Spend,
# Average cost per tap (CPT), Average cost per acquisition (CPA),
# Average Cost per Thousand-Impressions (CPM), Impressions, Taps, Installs,
# The tap-through rate (TTR), Conversion Rate (CR), LAT On Installs, LAT Off Installs,
# New Downloads, and Redownloads are of appropriate numeric data types.
numeric_cols = ['Lifetime Budget', 'Daily Budget', 'Spend', 'Average cost per tap (CPT)',
                'Average cost per acquisition (CPA)', 'Average Cost per Thousand-Impressions (CPM)',
                'Impressions', 'Taps', 'Installs', 'Tap-through rate (TTR)', 'Conversion Rate (CR)',
                'LAT On Installs', 'LAT Off Installs', 'New Downloads', 'Redownloads']

for col in numeric_cols:
  if col in df.columns:  # Check if the column exists
    try:
        # Attempt to convert the column to numeric, coercing errors to NaN
        df[col] = pd.to_numeric(df[col], errors='coerce')
        print(f"Column '{col}' successfully converted to numeric.")
    except Exception as e:
        print(f"Error converting '{col}' to numeric: {e}")
  else:
    print(f"Warning: Column '{col}' not found in DataFrame.")

# Now check for missing values again after numeric conversions
print(df.isnull().sum())


Start Date column successfully converted to datetime.
Column 'Lifetime Budget' successfully converted to numeric.
Column 'Daily Budget' successfully converted to numeric.
Column 'Spend' successfully converted to numeric.
Column 'Average cost per tap (CPT)' successfully converted to numeric.
Column 'Average cost per acquisition (CPA)' successfully converted to numeric.
Column 'Average Cost per Thousand-Impressions (CPM)' successfully converted to numeric.
Column 'Impressions' successfully converted to numeric.
Column 'Taps' successfully converted to numeric.
Column 'Installs' successfully converted to numeric.
Column 'Conversion Rate (CR)' successfully converted to numeric.
Column 'LAT On Installs' successfully converted to numeric.
Column 'LAT Off Installs' successfully converted to numeric.
Column 'New Downloads' successfully converted to numeric.
Column 'Redownloads' successfully converted to numeric.
Campaign ID                                    0
Campaign Name                     

In [55]:
# Campaign ID Consistency Check
duplicate_campaign_ids = df[df.duplicated(subset=['Campaign ID'], keep=False)]
if not duplicate_campaign_ids.empty:
    print("Duplicate Campaign IDs found:")
    print(duplicate_campaign_ids[['Campaign ID', 'Campaign Name']])

# Check for mismatches between Campaign ID and Campaign Name (if applicable)
# ... (add your logic to check for mismatches)


# Date Consistency Check
# Check for invalid or out-of-range Start Dates.  We already converted to datetime.
# Check for dates outside of a reasonable range (you'll need to define what's "reasonable").

min_valid_date = pd.to_datetime('2020-01-01')  # Example minimum valid date
max_valid_date = pd.to_datetime('2024-12-31')  # Example maximum valid date

invalid_dates = df[(df['Start Date'] < min_valid_date) | (df['Start Date'] > max_valid_date)]

if not invalid_dates.empty:
  print("Invalid or Out-of-range Start Dates:")
  print(invalid_dates[['Campaign ID', 'Campaign Name', 'Start Date']])



Duplicate Campaign IDs found:
    Campaign ID                 Campaign Name
1   481689442.0  0.54 - Tier 1 & 2 - May 2022
13  481689442.0                Top Tier Sleep


In [56]:
# # Campaign ID Consistency Check (Improved)
# duplicate_campaign_ids = df[df.duplicated(subset=['Campaign ID'], keep=False)]

# if not duplicate_campaign_ids.empty:
#     print("Duplicate Campaign IDs found:")
#     print(duplicate_campaign_ids[['Campaign ID', 'Campaign Name']])

#     # Group by 'Campaign ID' and get a list of corresponding 'Campaign Name' values
#     campaign_id_names = duplicate_campaign_ids.groupby('Campaign ID')['Campaign Name'].apply(list).reset_index()

#     # Print the Campaign ID and the list of associated Campaign Names
#     for index, row in campaign_id_names.iterrows():
#         campaign_id = row['Campaign ID']
#         names = row['Campaign Name']
#         print(f"\nCampaign ID: {campaign_id}")
#         for name in names:
#             print(f"  - {name}")


In [57]:

# Campaign ID Consistency Check (Improved)
duplicate_campaign_ids = df[df.duplicated(subset=['Campaign ID'], keep=False)]

if not duplicate_campaign_ids.empty:
    print("Duplicate Campaign IDs found:")
    print(duplicate_campaign_ids[['Campaign ID', 'Campaign Name']])

    # Group by 'Campaign ID' and get a list of corresponding 'Campaign Name' values
    campaign_id_names = duplicate_campaign_ids.groupby('Campaign ID')['Campaign Name'].apply(list).reset_index()

    # Print the Campaign ID and the list of associated Campaign Names,
    # and flag potential issues based on name similarity.
    for index, row in campaign_id_names.iterrows():
        campaign_id = row['Campaign ID']
        names = row['Campaign Name']
        print(f"\nCampaign ID: {campaign_id}")

        # Check for similar names (case-insensitive)
        for i in range(len(names)):
          for j in range(i + 1, len(names)):
            if names[i].lower() == names[j].lower():
                print(f"  - {names[i]} (Duplicate name)")
            elif names[i].lower() in names[j].lower() or names[j].lower() in names[i].lower():
                print(f"  - {names[i]} (Similar name to {names[j]})")
            else:
              print(f"  - {names[i]}")


Duplicate Campaign IDs found:
    Campaign ID                 Campaign Name
1   481689442.0  0.54 - Tier 1 & 2 - May 2022
13  481689442.0                Top Tier Sleep

Campaign ID: 481689442.0
  - 0.54 - Tier 1 & 2 - May 2022


In [58]:

# Range and Value Checks
def check_budget_spend(df):
    """
    Checks Lifetime Budget, Daily Budget, and Spend for non-negative values and reasonable limits.
    """
    for col in ['Lifetime Budget', 'Daily Budget', 'Spend']:
        if col in df.columns:
            # Check for negative values
            negative_values = df[df[col] < 0]
            if not negative_values.empty:
                print(f"\n Warning: Negative values found in '{col}' column:\n{negative_values[[col]]}")
                # Clip negative values to 0
                df[col] = df[col].clip(lower=0)
                print(f"Negative values in '{col}' clipped to 0.\n")

            # Check for excessively high values
            high_values = df[df[col] > 1000000]
            if not high_values.empty:
                print(f"\n Warning: Excessively high values found in '{col}' column:\n{high_values[[col]]}")
                # Clip high values to 1,000,000
                df[col] = df[col].clip(upper=1000000)
                print(f"High values in '{col}' clipped to 1,000,000.\n")

def check_performance_metrics(df):
    """
    Checks performance metrics for validity.
    """
    metrics = {
        'Average cost per tap (CPT)': (0, 100),
        'Average cost per acquisition (CPA)': (0, 500),
        'Average Cost per Thousand-Impressions (CPM)': (0, 50),
        'Tap-through rate (TTR)': (0, 100),
        'Conversion Rate (CR)': (0, 100)
    }

    for col, (min_val, max_val) in metrics.items():
        if col in df.columns:
            # Check for out-of-range values
            out_of_range = df[(df[col] < min_val) | (df[col] > max_val)]
            if not out_of_range.empty:
                print(f"\n Warning: Values outside the expected range found for '{col}':\n{out_of_range[[col]]}")
                # Clip out-of-range values
                df[col] = df[col].clip(lower=min_val, upper=max_val)
                print(f" Out-of-range values in '{col}' clipped to range ({min_val}, {max_val}).\n")

# Call the check functions
check_budget_spend(df)
check_performance_metrics(df)

# Display updated DataFrame
print("\n Updated DataFrame:\n", df.head())



 Updated DataFrame:
    Campaign ID                                  Campaign Name Start Date  \
0  470311161.0                         0.54 - May 2022 - Arab 2020-08-31   
1  481689442.0                   0.54 - Tier 1 & 2 - May 2022 2020-10-04   
2  469927825.0                       0.54 - Tier 1 - May 2022 2020-08-29   
3  474269632.0                             1.00 - My Keywords 2020-09-14   
4  484006307.0  1.01 - US/OZ - May 2022 - Performing Keywords 2020-10-12   

   Status                    App Name              Ad Placement  \
0  PAUSED  Sleep Habits: Sleep Better  App Store Search Results   
1  PAUSED  Sleep Habits: Sleep Better  App Store Search Results   
2  PAUSED  Sleep Habits: Sleep Better  App Store Search Results   
3  PAUSED  Sleep Habits: Sleep Better  App Store Search Results   
4  PAUSED  Sleep Habits: Sleep Better  App Store Search Results   

   Lifetime Budget  Daily Budget   Spend  Average cost per tap (CPT)  ...  \
0           5000.0         500.0  103.98 

In [59]:
# Logical Consistency
# Impressions, Taps, and Installs: Verify logical relationships such as Taps should not exceed Impressions, and Installs should not exceed Taps.
# LAT On/Off Installs and Downloads: Ensure that LAT On Installs and LAT Off Installs sum up correctly to the total number of Installs.

def check_logical_consistency(df):
    """
    Checks for logical inconsistencies in the data, such as:
    - Taps should not exceed Impressions.
    - Installs should not exceed Taps.
    - LAT On Installs + LAT Off Installs == Installs
    """

    # 1. Taps vs. Impressions
    invalid_taps = df[df['Taps'] > df['Impressions']]
    if not invalid_taps.empty:
        print("Warning: Taps exceed Impressions in some rows:")
        print(invalid_taps[['Campaign ID', 'Impressions', 'Taps']])
        # Handle the inconsistencies (e.g., clip Taps to Impressions)
        df['Taps'] = np.minimum(df['Taps'], df['Impressions'])
        print("Taps clipped to Impressions.")


    # 2. Installs vs. Taps
    invalid_installs = df[df['Installs'] > df['Taps']]
    if not invalid_installs.empty:
        print("\nWarning: Installs exceed Taps in some rows:")
        print(invalid_installs[['Campaign ID', 'Taps', 'Installs']])
        # Handle the inconsistencies (e.g., clip Installs to Taps)
        df['Installs'] = np.minimum(df['Installs'], df['Taps'])
        print("Installs clipped to Taps.")

    # 3. LAT On/Off Installs vs. Total Installs
    df['LAT_Combined_Installs'] = df['LAT On Installs'] + df['LAT Off Installs']
    install_mismatch = df[df['Installs'] != df['LAT_Combined_Installs']]
    if not install_mismatch.empty:
        print("\nWarning: LAT On/Off Installs do not sum up to Total Installs in some rows:")
        print(install_mismatch[['Campaign ID', 'Installs', 'LAT On Installs', 'LAT Off Installs', 'LAT_Combined_Installs']])

        # Option 1: Adjust LAT On/Off to match Total Installs (if you trust the 'Installs' column)
        # df['LAT On Installs'] = df.apply(lambda x: x['Installs'] - x['LAT Off Installs'], axis=1)

        # Option 2: Adjust Total Installs to match LAT On/Off (if you trust LAT On/Off)
        df['Installs'] = df['LAT_Combined_Installs']

        print("Installs adjusted to match the sum of LAT On/Off Installs.")

    # Drop the temporary 'LAT_Combined_Installs' column
    df = df.drop(columns=['LAT_Combined_Installs'], errors='ignore')  #errors='ignore' handles case if column doesn't exist

# Call the function
check_logical_consistency(df)



     Campaign ID  Installs  LAT On Installs  LAT Off Installs  \
0   4.703112e+08       664                7               337   
1   4.816894e+08        42                1                 3   
2   4.699278e+08       646               85               313   
3   4.742696e+08       444               67               171   
4   4.840063e+08       472               35               123   
5   4.870115e+08       215                2                 5   
6   5.798608e+08         3                0                 0   
7   4.814068e+08        14                0                 1   
9   1.153490e+09        16                0                 0   
11  4.698193e+08       503               78               290   
12  4.706172e+08       118               14                87   
13  4.816894e+08      3137              289              1330   

    LAT_Combined_Installs  
0                     344  
1                       4  
2                     398  
3                     238  
4            

In [60]:
# # Print the result now
# print(df.head())

In [61]:
# Check for duplicates in the dataset
print(df.duplicated().sum())

0


In [62]:
# Save the cleaned DataFrame to a new CSV file in the same directory
df.to_csv('drive/MyDrive/Data Analysis with Python/cleaned_apple_search_ads.csv', index=False)
